#  chatbot using LangGraph, LangChain, and Ollama

In [1]:
!pip install langgraph

  Using cached langgraph-0.3.21-py3-none-any.whl.metadata (7.7 kB)
  Using cached langchain_core-0.3.49-py3-none-any.whl.metadata (5.9 kB)
  Using cached langgraph_checkpoint-2.0.23-py3-none-any.whl.metadata (4.6 kB)
  Using cached langgraph_prebuilt-0.1.7-py3-none-any.whl.metadata (5.0 kB)
  Using cached langgraph_sdk-0.1.60-py3-none-any.whl.metadata (1.8 kB)
  Using cached langsmith-0.3.19-py3-none-any.whl.metadata (15 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached pydantic-2.11.0-py3-none-any.whl.metadata (63 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
Using cached langgraph-0.3.21-py3-none-any.whl (138 kB)
Using cached langchain_core-0.3.49-py3-none-any.whl (420 kB)
Using cached langgraph_checkpoint

In [2]:
from typing import List, Dict
from dataclasses import dataclass, field
from langgraph.graph import StateGraph, START, END
# from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Ollama


# Step 1: Define State using a dataclass
@dataclass
class State:
    messages: List[Dict[str, str]] = field(default_factory=list)


# Step 2: Initialize StateGraph
graph_builder = StateGraph(State)

# Initialize the LLM (using Llama 3.2)
llm = Ollama(model="llama3.2")


# Define chatbot function
def chatbot(state: State) -> Dict[str, List[Dict[str, str]]]:
    try:
        response = llm.invoke(state.messages)
        state.messages.append({"role": "assistant", "content": response})  # Treat response as a string
        return {"messages": state.messages}
    except Exception as e:
        return {"messages": [{"role": "assistant", "content": f"Error: {str(e)}"}]}


# Add nodes and edges
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Compile the graph
graph = graph_builder.compile()


# Stream updates
def stream_graph_updates(user_input: str):    
    state = State(messages=[{"role": "user", "content": user_input}])
    
    try:
        for event in graph.stream(state):
            for value in event.values():
                print("Assistant:", value["messages"][-1]["content"])
    except Exception as e:
        print(f"An error occurred: {e}")


# Run chatbot in a loop
if __name__ == "__main__":
    print("Chatbot started! Type 'quit', 'exit', or 'q' to stop.")

    while True:
        try:
            user_input = input("User: ").strip()
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            stream_graph_updates(user_input)

        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            break


C:\Users\konde\AppData\Local\Temp\ipykernel_7740\2576623463.py:18: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


Chatbot started! Type 'quit', 'exit', or 'q' to stop.


User:  hi


Assistant: Hello! How can I assist you today?


User:  what is astro physics.


Assistant: Astrophysics is the branch of astronomy that deals with the physical nature of celestial objects and phenomena. It's a field that seeks to understand the behavior, properties, and evolution of stars, galaxies, black holes, and other objects in the universe.

Astrophysicists use a combination of mathematical models, computational simulations, and observational data from telescopes and spacecraft to study the physical processes that govern the universe. This can include topics such as:

1. Stellar physics: The study of the properties and behavior of stars, including their structure, evolution, and interactions.
2. Galactic dynamics: The study of the behavior of galaxies, including their rotation curves, star clusters, and galaxy collisions.
3. Cosmology: The study of the origin, evolution, and fate of the universe as a whole.
4. Planetary science: The study of planets, moons, asteroids, comets, and other objects in our solar system.
5. High-energy astrophysics: The study of hi

User:  what is astral projection


Assistant: Astral projection, also known as out-of-body experience (OBE), is a phenomenon where an individual's consciousness or spirit temporarily leaves their physical body, allowing them to perceive the world from a non-physical perspective. During an OBE, the person may feel themselves floating above their body, observing their surroundings, and potentially interacting with other beings or energies.

The concept of astral projection has been explored in various cultures and spiritual traditions for centuries. Some believe that it's a natural ability that can be developed through meditation, visualization, and other practices. Others see it as a paranormal phenomenon that requires special skills or abilities.

During an OBE, individuals may report experiencing a range of sensations, including:

1. Feeling detached from their physical body
2. Seeing themselves and their surroundings from above
3. Observing events in slow motion or from multiple perspectives
4. Experiencing emotions a

User:  q


Goodbye!
